# Introduction to Keys and Signatures 

This is a short introduction to digital signatures by [Francesco Roda](mailto:francesco.roda@citi.com).

**⚠️Use in conjunction with the training slides ⚠️**



**The code**    
We define a few functions to generate a SECP256K1 key pair and sign a message with the private key and verify the signature with the public key. We will not encode the public key to derive a bitcoin and ethereum address.

In [2]:
#%pip install -q ecdsa
import ecdsa 
import binascii

def generate_keys():
    kpr = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    private_key = kpr.to_string().hex()
    kpu = kpr.get_verifying_key() 
    public_key = kpu.to_string().hex()
    return public_key, private_key, kpr
def derive_public_key(private_key_asc):
    #kpr_bytes =codecs.decode(private_key_asc, 'hex') #back to bytes
    kpr_bytes = binascii.unhexlify(private_key_asc)
    kpu = ecdsa.SigningKey.from_string(kpr_bytes, curve=ecdsa.SECP256k1).verifying_key
    #kpu_hex = '04' + codecs.encode(kpu.to_string(), 'hex').decode("utf-8")  #back to string
    kpu_hex = '04' + binascii.b2a_hex(kpu.to_string()).decode("utf-8")
    return kpu_hex
def sign_message(message, private_key): 
    bmessage = message.encode()
    sk = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    signature = sk.sign(bmessage)
    return signature, message
def verify_signature(public_key, signature, message): 
    vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(public_key), curve=ecdsa.SECP256k1)
    try: 
      vk.verify(signature, message.encode())
      return True 
    except:
      return False

**Create a key pair**  
We use the generate_key() function to randomly generate a private key and derive a public key.  
The sender of a message will use the private key to digitally sign a message. The recipient will use the public key of the sender to verify the integrity of a message.  
The sender publishes his public key, the message and the digital signature. Of course the sender does not publish the private key. 

In [3]:
public_key, private_key, private_key_u = generate_keys()
print("Public key:", public_key)
print("Private key:", private_key)


Public key: c16e5902268067a445631f7196c51b370c51d3b333b514079ed7c5dea9bc9c206d2a6210fe5045d9833c72d5cb9e6a9fd092cbc82bcc6812bbc7e39d648b2280
Private key: 7db4bd874afaf4dbf3632aa6b0b4ab575463ae0cb45386accbdae4f940245907


**The sender uses his private key to sign a message**  
We use the sign_message() function to sign a message with the private key.

In [4]:
signature, message = sign_message("I owe John 20 pounds, Sincerely Francesco Roda", private_key)
print("Message: ", message)
print("Signature: ",signature.hex())

Message:  I owe John 20 pounds, Sincerely Francesco Roda
Signature:  68c665343c73ba081743a3b1cdd5524b82b3d818c9160abd43a10db0111d1e38de608bd6269a3549569f65944fc64745ddfd54dde9d205e0e41f0d6a5f30cfa2


**The recipient uses the sender public key to verify the integrity of the message**  
We use the verify_signature() function to check the signature for a given message and a given private key.

In [5]:
test = verify_signature(public_key, signature, message) #this is what miners do 
print("The digital signature proves the integrity of the message", test)

The digital signature proves the integrity of the message True


**The recipient uses the sender public key to verify the integrity of another message with the same digital signature**
We use the verify_signature() function to check the signature for a given message and a given private key.

In [6]:
message_changed = "I owe John 19 pounds, Sincerely Francesco Roda"
print("New message: ", message_changed)
test = verify_signature(public_key, signature, message_changed)
print("The digital signature proves the integrity of the message", test) 

New message:  I owe John 19 pounds, Sincerely Francesco Roda
The digital signature proves the integrity of the message False
